# Import

In [13]:
import numpy as np
from fatiando import mesher, gridder, utils
from scipy.linalg import toeplitz
from sys import getsizeof
from timeit import default_timer as time
%matplotlib inline

# Observation and Equivalent layer grids

In [16]:
# Create a regular grid at 0m height
#area = [1, 4, 1, 3]
area = [0, 100, 0, 50]
shape = (100, 100)
xi, yi, zi = gridder.regular(area, shape, z=0.)

# Equivalent Layer
xj, yj, zj = gridder.regular(area, shape, z=80)

# Create the true parameters vector
N = shape[0]*shape[1]
p_true = np.arange(0,N,1)
p_true2 = p_true.reshape(N,1)

# Classic Forward Problem Construction - GRAV

In [17]:
# Calculo da matriz de sensibilidade
A = np.zeros((N, N), dtype=np.float)

s = time()
c_1 = zj[0]-zi[0]
c_2 = c_1*c_1
for i in xrange (N):
    a = (xj-xi[i])
    b = (yj-yi[i])
    A[i] = (a*a+b*b+c_2)**(-1.5)
A *= c_1

dobs_classic = A.dot(p_true2)

e = time()
tcpu = e - s
print tcpu
print A.nbytes/(1024.*1024.)

5.89216416357
762.939453125


# First Row to Toeplitz Blocks - Forward Model

In [18]:
# Simplified allocation of blocks*parameters (slower)

s = time()
dobs_bt = np.zeros(N).reshape(N,1)

l = shape[0]

for i in range (shape[1]): 
    k = 0 # index of the parameter's segment (lower matrix)/ index of the data's segment (upper matrix)
    j = i # index of the parameter's segment (upper matrix)/ index of the data's segment (lower matrix)

    a = (xj[shape[0]*(i):shape[0]*(i+1)]-xi[0])
    b = (yj[shape[0]*(i):shape[0]*(i+1)]-yi[0])
    c = (zj[shape[0]*(i):shape[0]*(i+1)]-zi[0])
    W_bt = c/(a*a+b*b+c*c)**(1.5)
    block = toeplitz(W_bt) # create each Toeplitz block by the segment of the first row
        
    while k < l:
        
        # lower matrix
        block_p = block.dot(p_true2[shape[0]*(k):shape[0]*(k+1)])
        dobs_bt[shape[0]*(j):shape[0]*(j+1)] += block_p
        
        # upper matrix
        if i > 0:
            block_p = block.dot(p_true2[shape[0]*(j):shape[0]*(j+1)])
            dobs_bt[shape[0]*(k):shape[0]*(k+1)] += block_p
        
        k += 1
        j += 1
    l -= 1
    
e = time()
tcpu = e - s
print tcpu
print block.nbytes/(1024.*1024.)

0.0814180707428
0.0762939453125


In [19]:
# More complex allocation of blocks*parameters (faster)

s = time()
dobs_bt_1 = np.zeros(N).reshape(N,1)

l = shape[0]

for i in range (shape[1]): 
    k = 0 # index of the parameter's segment
    j = 0+i # index of the data's segment using the lower matrix
    u = 0 # index of the data's segment using the upper matrix
    a = (xj[shape[0]*(i):shape[0]*(i+1)]-xi[0])
    b = (yj[shape[0]*(i):shape[0]*(i+1)]-yi[0])
    c = (zj[shape[0]*(i):shape[0]*(i+1)]-zi[0])
    W_bt = c/(a*a+b*b+c*c)**(1.5)
    block = toeplitz(W_bt) # create each Toeplitz block by the segment of the first row
        
    while k < l: # lower matrix
        block_p = block.dot(p_true2[shape[0]*(k):shape[0]*(k+1)])
        dobs_bt_1[shape[0]*(j):shape[0]*(j+1)] += block_p
        if k >= i and i > 0: # upper matrix 1
            dobs_bt_1[shape[0]*(u):shape[0]*(u+1)] += block_p
            u += 1
        k += 1
        j += 1
    while i > 0 and k >= l and k < shape [1]: # upper matrix 2
        if k >= i:
            block_p = block.dot(p_true2[shape[0]*(k):shape[0]*(k+1)])       
            dobs_bt_1[shape[0]*(u):shape[0]*(u+1)] += block_p
            u += 1
        k += 1
    l -= 1
    
e = time()
tcpu = e - s
print tcpu
print block.nbytes/(1024.*1024.)

0.0546929659254
0.0762939453125


In [20]:
np.allclose(dobs_classic, dobs_bt, dobs_bt_1)

True